In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import nltk
from nltk.stem import PorterStemmer
porter = PorterStemmer()
from nltk.corpus import wordnet
nltk.download('omw-1.4')
import numpy as np
from sklearn.model_selection import train_test_split

import pickle

from numpy import mean
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from imblearn.pipeline import Pipeline
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN



[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\qhuca\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')

data = dataset[0]

In [3]:
features = data.x.numpy()
labels = data.y.numpy()

In [14]:
X = features
y = labels
# define pipeline
steps = [('over', RandomOverSampler()), ('model', BalancedRandomForestClassifier())]
pipeline = Pipeline(steps=steps)
# evaluate pipeline
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
score = mean(scores)
print('F1 Score: %.3f' % score)
# all_scores.append(score)


F1 Score: 0.770


In [15]:
from sklearn.ensemble import RandomForestClassifier
X = features
y = labels
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
cross_val_score(RandomForestClassifier(), X, y, scoring='accuracy', cv=cv, n_jobs=-1).mean()

0.7659938043824882

In [16]:
from sklearn.linear_model import LogisticRegression
X = features
y = labels
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=1)
cross_val_score(LogisticRegression(), X, y, scoring='accuracy', cv=cv, n_jobs=-1).mean()

0.7719689763564302

In [7]:
# from sklearn.svm import SVC
# X = features
# y = labels
# cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# cross_val_score(SVC(), X, y, scoring='accuracy', cv=cv, n_jobs=-1).mean()

In [20]:
import lightgbm as lgb
from xgboost import XGBClassifier
X = features
y = labels
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=1)
cross_val_score(XGBClassifier(), X, y, scoring='accuracy', cv=cv, n_jobs=-1).mean()
# XGBClassifier()
# lgb.LGBMClassifier()

0.7721541394335512

In [22]:
pd.Series(data.y.numpy()).value_counts()

3    818
4    426
2    418
0    351
5    298
1    217
6    180
dtype: int64

In [3]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)
    


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    # out = F.log_softmax(out, dim=1)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    


In [5]:
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.8000


In [6]:
# model(data).shape
features2 = model(data).cpu().detach().numpy()

In [8]:
data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [18]:
# X = np.concatenate((data.x.cpu().numpy(), features2),1)
X = data.x.cpu().numpy()
y = data.y.cpu().numpy()

X_train = X[data.train_mask.cpu()]
y_train = y[data.train_mask.cpu()]

X_val = X[data.val_mask.cpu()]
y_val = y[data.val_mask.cpu()]

X_test = X[data.test_mask.cpu()]
y_test = y[data.test_mask.cpu()]

import numpy as np
X = np.concatenate((X_train, X_val, X_test),0)
y = np.concatenate((y_train, y_val, y_test),0)

In [19]:
# import lightgbm as lgb
# from xgboost import XGBClassifier
# cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=1)
# cross_val_score(lgb.LGBMClassifier(), X, y, scoring='accuracy', cv=cv, n_jobs=-1).mean()
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from imblearn.pipeline import Pipeline
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
# define pipeline

steps = [('over', RandomOverSampler()), ('model', BalancedRandomForestClassifier())]
pipeline = Pipeline(steps=steps)
# evaluate pipeline
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
scores = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
score = mean(scores)
print('classification Score: %.3f' % score)

classification Score: 0.738


In [14]:
import lightgbm as lgb
from sklearn import metrics
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
X_train, y_train = ros.fit_resample(X_train, y_train)

model = lgb.LGBMClassifier()
model.fit(X_train, y_train)
expected_y  = y_test
predicted_y = model.predict(X_test)
# print(metrics.classification_report(expected_y, predicted_y))

In [15]:
metrics.accuracy_score(expected_y, predicted_y)

0.331